In [1]:
# imports for this notebook
from pandas import *
import numpy as np
import matplotlib.pyplot as plt
plt.ion()
%config InlineBackend.figure_format='retina'

# yotam's CANDELizing pipeline for SAM mock catalog

In this notebook, we demonstrate how to take a lightcone catalog file generated by the Somerville+ SAM and run it through our "CANDELizing" pipeline. The pipeline has several goals including correcting the mock catalog for CANDELS completeness limitations, correcting galaxy sizes, and adding photometric noise.

Dependencies:
* python (2.7+ probably best)
* NumPy (http://www.numpy.org/)
* Pandas (http://pandas.pydata.org/)
* PyTables (http://www.pytables.org/)
* H5pi (http://www.h5py.org/)
* CosmoloPy (http://roban.github.io/CosmoloPy/)


You should also upgrade all those packages (except python, best to use 2.7) to their latest versions to avoid incompatibility issues. This can be accomplishes with pip:

    pip install --upgrade numpy
    pip install --upgrade pandas
    pip install --upgrade tables
    pip install --upgrade h5py
    pip install --upgrade cosmolopy

# Download

First, download the pipeline repository, if you haven't already, and move into that directory. It can be found at:

https://github.com/yotamcohen/CANDELizing_pipeline

Lets see whats inside:

In [2]:
%%bash
ls -F

CANDELizing_pipeline_tutorial.ipynb
example_lightcone.dat
example_lightcone.h5
lib/
plots/
python_scripts/
readme.txt
runpipeline.sh*


Okay, so we see directories containing library files and python scripts, and an example lightcone file called example_lightcone.dat, which we will use in this demonstration. This example lightcone file is a subset of a full lightcone.dat file from a GOODS-S realization, trimmed down to a managable size for example purposes. You can also use it for testing the pipeline on your own machine. The lightcone file is expected to be in the particular format as the example one we have here, complete with the header rows and all of the columns output by the SAM code. The version of the SAM code used for this paper writes 142 columns to the lightcone.dat file by default:

In [3]:
%%bash
head -n 142 example_lightcone.dat

# 0 halo_id_nbody
# 1 gal_id
# 2 gal_type (0, 1, 2)
# 3 z_nopec [] (distance redshift w/o pec velocity)
# 4 redshift []
# 5 ra [degree]
# 6 dec [degree]
# 7 m_vir [1.0E10 M_sun]
# 8 V_vir [km/s]
# 9 r_vir [Mpc]
# 10 c_NFW []
# 11 spin []
# 12 mstar_diffuse [1.0E10 M_sun]
# 13 m_hot_halo [1.0E10 M_sun]
# 14 Z_hot_halo [Z_sun]
# 15 v_disk [km/s]
# 16 r_disk [kpc]
# 17 sigma_bulge [km/s]
# 18 rbulge [kpc]
# 19 mhalo [1.0E10 M_sun]
# 20 mstar [1.0E10 M_sun]
# 21 mcold [1.0E10 M_sun]
# 22 mbulge [1.0E10 M_sun]
# 23 mbh [1.0E10 M_sun]
# 24 maccdot [M_sun/yr]
# 25 maccdot_radio [M_sun/yr]
# 26 Zstar [Z_sun]
# 27 Zcold [Z_sun]
# 28 mstardot [M_sun/yr]
# 29 sfr_ave d [M_sun/yr]
# 30 meanage [Gyr]
# 31 tmerge [Gyr]
# 32 tmajmerge [Gyr]
# 33 cosi []
# 34 UV1500_rest CANDELS/TOPHAT/UV1500top.dat
# 35 UV1500_rest_bulge CANDELS/TOPHAT/UV1500top.dat
# 36 UV1500_rest_dust CANDELS/TOPHAT/UV1500top.dat
# 37 UV2300_rest CANDELS/TOPHAT/UV2300top.dat
# 38 UV2300_rest_bulge CANDELS/TOPHAT/UV2300top.dat
# 39

# wrapper

There is a wrapper included called `runpipeline.sh`. If you want to attempt to run the whole pipeline at once, just go inside this file, set the path to the working directory (which should be the directory containing the pipeline), the path to the original lightcone file, and the path to the (soon to be created) h5file. Then, just run the wrapper. To see how the individual steps work, continue reading on.

In [15]:
%%bash
./runpipeline.sh

grabbed lightcone of shape (12158, 142)
writing to h5 file...
saved lightcone data to /Users/yotam/CANDELizing_pipeline/example_lightcone.h5
all units are same as in original lightcone data file
grabbed dataframe. working...
saved dataframe to /Users/yotam/CANDELizing_pipeline/example_lightcone.h5
grabbed dataframe from /Users/yotam/CANDELizing_pipeline/example_lightcone.h5
dataframe now includes bulge/disk ratio from projected sizes
dataframe now includes sersic profile parameters
dataframe now includes galaxy radii in PIXELS as seen by HSTWFC3F160W
saved dataframe to /Users/yotam/CANDELizing_pipeline/example_lightcone.h5
grabbed lightcone
grabbed photometry from CANDELS
acs_f435w_mag (37394, 308)
acs_f606w_v08_mag (8208, 308)
acs_f775w_mag (40288, 308)
acs_f814w_v08_mag (8224, 308)
acs_f850lp_mag (40777, 308)
wfc3_f275w_mag (4885, 308)
wfc3_f105w_mag (33926, 308)
wfc3_f125w_v08_mag (8215, 308)
wfc3_f160w_v08_mag (8236, 308)
irac_ch1_mag (23652, 308)
irac_ch2_mag (23629, 308)
datafram

python_scripts/photometric_noise_new.py:82: RuntimeWarning: invalid value encountered in log10
  mdf[mmagnoisy] = -2.5*np.log10(mdf[mfluxnoisy]) + 23.9
python_scripts/photometric_noise_new.py:85: RuntimeWarning: invalid value encountered in log10
  mdf[mmagdustnoisy] = -2.5*np.log10(mdf[mfluxdustnoisy]) + 23.9
python_scripts/calculate_dp.py:32: RuntimeWarning: divide by zero encountered in true_divide
  Hd[np.isneginf(1/Hd)] = np.nan
python_scripts/calculate_dp.py:33: RuntimeWarning: divide by zero encountered in true_divide
  He[np.isneginf(1/He)] = np.nan


# Step 1: ascii text file to hdf5 file

The native format of the lightcone.dat files output by the SAM is simple ASCII text, which can be very slow to read/write with python, so the first step is to store all the data in an hdf5 file. For this, we will use python_scripts/rawlightconefile_to_h5.py. This script is expecting a header file containing the native header rows, which should live in the lib/ directory; if it doesn't already exist, you can just do something like

In [4]:
%%bash
head -n 142 example_lightcone.dat > lib/headerfile.txt

To use the h5 conversion script from the command line (from this directory), just do:

    python python_scripts/rawlightconefile_to_h5.py <headerfile> <lightcone file> <desired path to h5file>

In [5]:
%%bash
python python_scripts/rawlightconefile_to_h5.py lib/headerfile.txt ./example_lightcone.dat ./example_lightcone.h5

grabbed lightcone of shape (12158, 142)
writing to h5 file...
saved lightcone data to ./example_lightcone.h5
all units are same as in original lightcone data file


In [6]:
%%bash
ls -ltrhF

total 115008
-rw-r--r--   1 yotam  staff    16M Jan 22 15:54 example_lightcone.dat
-rw-r--r--   1 yotam  staff   121B Jan 22 15:54 readme.txt
drwxr-xr-x  13 yotam  staff   442B Jan 22 15:54 python_scripts/
drwxr-xr-x   9 yotam  staff   306B Jan 22 15:54 plots/
drwxr-xr-x  12 yotam  staff   408B Jan 22 15:54 lib/
-rwxr-xr-x   1 yotam  staff   535B Jan 24 13:50 runpipeline.sh*
-rw-r--r--   1 yotam  staff    32K Jan 24 13:51 CANDELizing_pipeline_tutorial.ipynb
-rw-r--r--   1 yotam  staff    40M Jan 24 13:51 example_lightcone.h5


Great, so we see the file was created. Lets open it up to make sure everythings there. You can easily open it with pandas in an interactive python sessios (like this notebook). By default, the data will be stored under the 'dat' key:

In [7]:
store = HDFStore('example_lightcone.h5')
lcone = store['dat']
store.close()

print lcone.head()

   halo_id_nbody  gal_id  gal_type   z_nopec  redshift         ra        dec  \
0     2776575133       1         0  0.018547  0.020271  53.374077 -27.527531   
1     2691101762       1         0  0.074530  0.076570  52.987942 -27.819973   
2     2677376520      12         2  0.075058  0.077036  53.005903 -27.613504   
3     2661838289      16         2  0.085732  0.083417  53.351725 -27.662076   
4     2661851549       1         0  0.090251  0.089146  52.952389 -27.695505   

       m_vir      V_vir     r_vir      ...        UKIRT_H_dust    UKIRT_K  \
0   2.423304  37.088756  0.075312      ...           20.788730  21.144127   
1   1.485251  32.040132  0.061878      ...           25.996959  26.278141   
2  14.719764  68.831694  0.132876      ...           21.741104  21.965658   
3  10.148968  60.924214  0.116946      ...           23.278740  23.523828   
4   1.074189  28.868147  0.055132      ...           26.129287  26.398970   

   UKIRT_K_bulge  UKIRT_K_dust   irac_ch1  irac_ch1_bulg

So there's our lightcone data, now stored in h5 format for efficient reading/writing for the rest of the pipeline and just accessing the data in general for e.g. plotting, analysis, etc.

# Step 2: galaxy size corrections

The SAM code reports the 3D bulge and disk sizes. We want to also have the 2D (projected) sizes. There is a perscription for how to do this conversion, which is implemented in python_scripts/galaxy_size_corrections.py. The usage is simple, just call the script with the h5file from the previous step as the only argument:

    python python_scripts/galaxy_size_corrections.py <lightcone h5 file>

In [8]:
%%bash
python python_scripts/galaxy_size_corrections.py example_lightcone.h5

grabbed dataframe. working...
saved dataframe to example_lightcone.h5


In [9]:
store = HDFStore('example_lightcone.h5')
lcone = store['dat']
store.close()

print lcone.head()

   halo_id_nbody  gal_id  gal_type   z_nopec  redshift         ra        dec  \
0     2776575133       1         0  0.018547  0.020271  53.374077 -27.527531   
1     2691101762       1         0  0.074530  0.076570  52.987942 -27.819973   
2     2677376520      12         2  0.075058  0.077036  53.005903 -27.613504   
3     2661838289      16         2  0.085732  0.083417  53.351725 -27.662076   
4     2661851549       1         0  0.090251  0.089146  52.952389 -27.695505   

       m_vir      V_vir     r_vir    ...     irac_ch1_dust   irac_ch2  \
0   2.423304  37.088756  0.075312    ...         21.939049  22.395286   
1   1.485251  32.040132  0.061878    ...         27.071520  27.457822   
2  14.719764  68.831694  0.132876    ...         22.752746  23.196188   
3  10.148968  60.924214  0.116946    ...         24.317908  24.729002   
4   1.074189  28.868147  0.055132    ...         27.193208  27.582015   

   irac_ch2_bulge  irac_ch2_dust  r_disk_proj_star  r_bulge_proj_star  \
0      

As we can see, there are a few new columns at the end of the dataframe, including the projected bulge and disk sizes (in the original units of kpc) in H-band and stellar mass, as well as bulge/total ratios.

# Step 3: various other parameters

The main goal of this pipeline is to do a completeness correction on the mock catalogs given the completeness limits of CANDELS. People at STScI have generated lookup tables which give the probability of detecting a galaxy given it's H-band magnitude, effective radius in pixels (given the plate scale of HSTWFC3F160W), and sersic index. In order to use these, we need to calculate and add these parameters to our mock galaxy catalog.

* The effective radius and sersic index calculation is done using a model which takes as input the bulge/total ratio (based on light) and bulge size to disk size ratio.
* In order to calculate the size in pixels, we first need the angular size. The galaxies are far away, so the small angle approximation tells us that the angular size is just given by the physical size divided by the distance. Since our mock galaxies have large, cosmological redshifts, the distance must be calculated using a cosmology library (this is where CosmoloPy comes in). Once we know the distance, we calculate the angular size, and then we calculate the size in pixels simply by knowning the plate scale of HSTWFC3.

The above is implemented in python_scripts/galaxy_profiles.py. Usage is simple:

    python python_scripts/galaxy_profiles.py <lighcone h5 file>

(Beware that the sersic index calculation is slow and can take several minutes or even more than an hour for a full (>1Gb) lightcone.)


In [10]:
%%bash
python python_scripts/galaxy_profiles.py example_lightcone.h5

grabbed dataframe from example_lightcone.h5
dataframe now includes bulge/disk ratio from projected sizes
dataframe now includes sersic profile parameters
dataframe now includes galaxy radii in PIXELS as seen by HSTWFC3F160W
saved dataframe to example_lightcone.h5


Again, lets check the output...

In [11]:
store = HDFStore('example_lightcone.h5')
lcone = store['dat']
store.close()

print lcone.head()

   halo_id_nbody  gal_id  gal_type   z_nopec  redshift         ra        dec  \
0     2776575133       1         0  0.018547  0.020271  53.374077 -27.527531   
1     2691101762       1         0  0.074530  0.076570  52.987942 -27.819973   
2     2677376520      12         2  0.075058  0.077036  53.005903 -27.613504   
3     2661838289      16         2  0.085732  0.083417  53.351725 -27.662076   
4     2661851549       1         0  0.090251  0.089146  52.952389 -27.695505   

       m_vir      V_vir     r_vir     ...      r_bulge_proj_H  BT_light  \
0   2.423304  37.088756  0.075312     ...            0.002292  0.044357   
1   1.485251  32.040132  0.061878     ...            0.000002  0.062672   
2  14.719764  68.831694  0.132876     ...            0.081765  0.208211   
3  10.148968  60.924214  0.116946     ...            0.003835  0.041531   
4   1.074189  28.868147  0.055132     ...            0.000006  0.043981   

    BT_mass           RBD  n_sersic      reff            d_a        

As we can see, there are several new columns. Just to be clear, lets name them all here:

* RBD : ratio of projected bulge radius to projected disk radius
* n_sersic : sersic index
* reff: projected composite H-band radius in kpc
* d_a : cosmological angular diameter distance in kpc
* d_L : cosmological luminosity distance in kpc
* angular_radius : in arcseconds
* r_pixels : given the plate scale of HSTWFC3F160W of 0.06 arcsec/pixel

# Step 4: add photometric noise

We would now like to add noise to our mock galaxies photometry. One of the main reasons for doing this is because in the following step, we would like to estimate the detection probability at a given observed magnitude. Real observations in the survey are plagued by noise. Thus, we would like to replicate the effect of this noise, and then use the noisy photometry (as opposed to the clean photometry) as the input to our detection probability calculation. The systematics of HST and noise background in various bandpasses is well-understood, so we use this information in order to assign photometric noise to the mock photometry. This is accomplished by running python_scripts/photometric_noise.py. Usage is simple:

    python python_scripts/photometric_noise_new.py <lightcone h5 file>

In [12]:
%%bash
python python_scripts/photometric_noise_new.py example_lightcone.h5

grabbed lightcone
grabbed photometry from CANDELS
acs_f435w_mag (37394, 308)
acs_f606w_v08_mag (8208, 308)
acs_f775w_mag (40288, 308)
acs_f814w_v08_mag (8224, 308)
acs_f850lp_mag (40777, 308)
wfc3_f275w_mag (4885, 308)
wfc3_f105w_mag (33926, 308)
wfc3_f125w_v08_mag (8215, 308)
wfc3_f160w_v08_mag (8236, 308)
irac_ch1_mag (23652, 308)
irac_ch2_mag (23629, 308)
dataframe now includes photometric noise
saved to example_lightcone.h5


python_scripts/photometric_noise_new.py:82: RuntimeWarning: invalid value encountered in log10
  mdf[mmagnoisy] = -2.5*np.log10(mdf[mfluxnoisy]) + 23.9
python_scripts/photometric_noise_new.py:85: RuntimeWarning: invalid value encountered in log10
  mdf[mmagdustnoisy] = -2.5*np.log10(mdf[mfluxdustnoisy]) + 23.9


^^^ Those warnings are generated because a small fraction of the objects end up with a negative flux after adding the gaussian noise, so the magnitudes cannot be calculated and will appear as a NaN value.

# Step 5: detection probability estimation

We now have all the information needed to lookup the probability of detecting a given mock galaxy given the limitations of CANDELS and the properties of the galaxy. This is implemented in python_scripts/calculate_dp.py. Usage is as follows:

    python python_scripts/calculate_dp.py <lightcone h5 file> <table for disks> <table for ellipticals>

The lookup tables are in lib/completeness_tables. There are tables for each CANDELS field, and they are further split by the galaxy type. All you have to do is call the script with the lookup table files for the CANDELS field corresponding to your lightcone file. In this example, we have been using a subset of a GOODS-S lightcone file, so we will call those lookup tables:

In [13]:
%%bash
python python_scripts/calculate_dp.py example_lightcone.h5 \
./lib/completeness_tables/goodss_expdisk.npz \
./lib/completeness_tables/goodss_devauc.npz

working on example_lightcone.h5
dataframe now includes detection probabilities


python_scripts/calculate_dp.py:32: RuntimeWarning: divide by zero encountered in true_divide
  Hd[np.isneginf(1/Hd)] = np.nan
python_scripts/calculate_dp.py:33: RuntimeWarning: divide by zero encountered in true_divide
  He[np.isneginf(1/He)] = np.nan


Let's check it out...

In [14]:
store = HDFStore('example_lightcone.h5')
lcone = store['dat']
store.close()

print lcone.head()

   halo_id_nbody  gal_id  gal_type   z_nopec  redshift         ra        dec  \
0     2776575133       1         0  0.018547  0.020271  53.374077 -27.527531   
1     2691101762       1         0  0.074530  0.076570  52.987942 -27.819973   
2     2677376520      12         2  0.075058  0.077036  53.005903 -27.613504   
3     2661838289      16         2  0.085732  0.083417  53.351725 -27.662076   
4     2661851549       1         0  0.090251  0.089146  52.952389 -27.695505   

       m_vir      V_vir     r_vir   ...     irac_ch1_dust_mag_noisy  \
0   2.423304  37.088756  0.075312   ...                   21.910410   
1   1.485251  32.040132  0.061878   ...                   25.888894   
2  14.719764  68.831694  0.132876   ...                   22.792020   
3  10.148968  60.924214  0.116946   ...                   24.458672   
4   1.074189  28.868147  0.055132   ...                         NaN   

   irac_ch2_mag  irac_ch2_flux  irac_ch2_dust_mag  irac_ch2_dust_flux  \
0     22.395286    


There is one new column, called dp (detection probability). Now we can do whatever we want with this.